In [1]:
from google.colab import drive
drive.mount('/content/drive')
Work_path  ="/content/drive/MyDrive/ProjectWork/Files/"
Path_log_file = "/content/drive/MyDrive/ProjectWork/path_log.csv"

import pandas as pd
import os

df = pd.read_csv('/content/drive/MyDrive/ProjectWork/corpus_clean_EN.csv', sep=';', engine='c')
df.head()
print(df.shape)

Mounted at /content/drive
(511, 3)


In [2]:
# Install
!pip install pyLDAvis -qq
!pip install -qq -U gensim
!pip install spacy -qq
!pip install matplotlib -qq
!pip install seaborn -qq
!python -m spacy download en_core_web_md -qq

# Import
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()# Visualise inside a notebook
import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

# Read the data
reports = pd.read_csv('/content/drive/MyDrive/ProjectWork/corpus_clean_EN.csv', sep=';', engine='c')
reports.head()
reports.info()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.8 MB/s eta 0:00:00
2023-04-16 19:40:51.204413: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 19:40:52.214805: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 13.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


/usr/local/lib/python3.9/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.9/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Language   511 non-null    object
 1   Path       511 non-null    object
 2   CleanText  511 non-null    object
dtypes: object(3)
memory usage: 12.1+ KB


In [3]:
# Our spaCy model:
nlp = en_core_web_md.load()

# Tags I want to remove from the text
removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
tokens = []

for summary in nlp.pipe(reports['CleanText']):
   proj_tok = [token.lemma_.lower() for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha]
   tokens.append(proj_tok)

# Add tokens to new column
reports['tokens'] = tokens
reports['tokens']

# Create dictionary
# I will apply the Dictionary Object from Gensim, which maps each word to their unique ID:
dictionary = Dictionary(reports['tokens'])
print(dictionary.token2id)

# Filter dictionary
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)

# Create corpus
corpus = [dictionary.doc2bow(doc) for doc in reports['tokens']]

# # LDA model building
# lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=50, num_topics=10, workers = 4, passes=10)

# # Coherence score using C_umass:
# topics = []
# score = []
# for i in range(1,20,1):
#    lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=10, num_topics=i, workers = 4, passes=10, random_state=100)
#    cm = CoherenceModel(model=lda_model, corpus=corpus, dictionary=dictionary, coherence='u_mass')
#    topics.append(i)
#    score.append(cm.get_coherence())
# _=plt.plot(topics, score)
# _=plt.xlabel('Number of Topics')
# _=plt.ylabel('Coherence Score')
# plt.show()

# # Coherence score using C_v:
# topics = []
# score = []
# for i in range(1,20,1):
#    lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=30, num_topics=i, workers = 4, passes=10, random_state=100)
#    cm = CoherenceModel(model=lda_model, texts = reports['tokens'], corpus=corpus, dictionary=dictionary, coherence='c_v')
#    topics.append(i)
#    score.append(cm.get_coherence())
# _=plt.plot(topics, score)
# _=plt.xlabel('Number of Topics')
# _=plt.ylabel('Coherence Score')
# plt.show()

# Optimal model
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=100, num_topics=9, workers = 4, passes=100)

# Print topics
lda_model.print_topics(-1)

# Where does a text belong to
lda_model[corpus][0]
reports['CleanText'][0]

# Visualize topics
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)

# Save the report
pyLDAvis.save_html(lda_display, 'index.html')

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'ali': 0, 'appendix': 1, 'arjo': 2, 'associates': 3, 'basic': 4, 'bauer': 5, 'belgium': 6, 'business': 7, 'buy': 8, 'call': 9, 'cam': 10, 'case': 11, 'centre': 12, 'cheap': 13, 'com': 14, 'come': 15, 'company': 16, 'companyregistered': 17, 'consultant': 18, 'contact': 19, 'customer': 20, 'datum': 21, 'destroy': 22, 'detail': 23, 'discuss': 24, 'electrical': 25, 'email': 26, 'end': 27, 'england': 28, 'expect': 29, 'explain': 30, 'find': 31, 'floor': 32, 'folkestone': 33, 'follow': 34, 'general': 35, 'give': 36, 'glenmore': 37, 'go': 38, 'gsw': 39, 'hand': 40, 'hans': 41, 'hit': 42, 'hour': 43, 'house': 44, 'information': 45, 'instal': 46, 'install': 47, 'interim': 48, 'issue': 49, 'jmassociates': 50, 'john': 51, 'language': 52, 'limited': 53, 'linkedinlinkedin': 54, 'locate': 55, 'lot': 56, 'machine': 57, 'mail': 58, 'meeting': 59, 'mention': 60, 'mill': 61, 'mobile': 62, 'modification': 63, 'moldavia': 64, 'money': 65, 'month': 66, 'morris': 67, 'necessary': 68, 'need': 69, 'new': 70,

/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [4]:
from gensim import models
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(


In [5]:
def find_similar(doc, n):
  #doc = "reservation hotel travel"
  vec_bow = dictionary.doc2bow(doc.lower().split())
  vec_lsi = lsi[vec_bow]  # convert the query to LSI space
  #print(vec_lsi)

  from gensim import similarities
  index = similarities.MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index

  index.save('/tmp/deerwester.index')
  index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')

  sims = index[vec_lsi]  # perform a similarity query against the corpus
  #print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples

  documents = list(reports['CleanText'])
  results = []
  sims = sorted(enumerate(sims), key=lambda item: -item[1])
  for doc_position, doc_score in sims:
      #print(doc_score, documents[doc_position])
      results.append(documents[doc_position])

  tx = []
  for i in range(0,n):
    tx.append(results[i])

  tx1 = '|'.join(tx)

  df1 = reports[reports["CleanText"].str.lower().str.contains(tx1)]
  df1 = df1.drop_duplicates(subset=['CleanText'])
  print('---------------------------------------')
  print(' Articoli trovati: ' + str(len(df1)))
  print('---------------------------------------')

  save_path = '/content/drive/MyDrive/ProjectWork/Results/'
  if not os.path.exists(save_path):
    os.makedirs(save_path)

  for p in df1.Path:
    dst = save_path + os.path.basename(p).split('/')[-1]
    shutil.copyfile(p, dst)

  return(results)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
from IPython.display import HTML, display

from ipywidgets import interact, Layout, HBox, VBox, Box
import ipywidgets as widgets
from IPython.display import clear_output
import shutil



/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
def relevant_articles_for_text():    
    textW = widgets.Textarea(
        value='',
        placeholder='Type something',
        description='',
        disabled=False,
        layout=Layout(width='90%', height='100px')
    )

    yearW = widgets.IntRangeSlider(min=1950, max=2020, value=[2010, 2020], description='Year Range', 
                               continuous_update=False, layout=Layout(width='40%'))
    covidW = widgets.Checkbox(value=True,description='Only COVID-19-Papers',disabled=False, indent=False, layout=Layout(width='25%'))
    kWidget = widgets.IntSlider(value=10, description='n', max=100, min=1, layout=Layout(width='25%'))

    button = widgets.Button(description="Trova Report")

    #display(VBox([HBox([kWidget, yearW, covidW], layout=Layout(width='90%', justify_content='space-around')),
    display(VBox([HBox([kWidget], layout=Layout(width='90%', justify_content='space-around')),
        textW, button], layout=Layout(align_items='flex-start')))

    def on_button_clicked(b):
        clear_output()
        #display(VBox([HBox([kWidget, yearW, covidW], layout=Layout(width='90%', justify_content='space-around')),
        display(VBox([HBox([kWidget], layout=Layout(width='90%', justify_content='space-around')),
            textW, button], layout=Layout(align_items='flex-start')))        
        find_similar(textW.value, kWidget.value)
        #print(top_res)

    button.on_click(on_button_clicked)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
relevant_articles_for_text()

---------------------------------------
 Articoli trovati: 17
---------------------------------------
